In [1]:
import pandas as panda
import numpy as np
from collections import Counter

In [2]:
hist_with_merchants_data_path = 'C:/TV Series/real_world_use_cases_1/supervised/regression/kaggle/elo/all_new/new_with_no_merchant.csv'


In [3]:
hist_transaction_data = panda.read_csv(hist_with_merchants_data_path)
hist_transaction_data_types = hist_transaction_data.dtypes
hist_transaction_column_types = {}
for index, column_name in enumerate(hist_transaction_data.columns.tolist()):
    hist_transaction_column_types[column_name] = hist_transaction_data_types[index]
del hist_transaction_data
hist_transaction_data = panda.read_csv(hist_with_merchants_data_path, dtype = hist_transaction_column_types)
hist_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_8c9ba46f19,17,N,0,A,2,NaN,2,1.597228,2017-12-19 08:07:34,4.0,22,20
1,Y,C_ID_0a6ceeffcc,143,N,0,A,884,NaN,1,-0.596673,2017-11-24 00:00:00,5.0,5,27
2,Y,C_ID_d15cd87e37,73,N,0,A,108,NaN,1,-0.296112,2018-03-28 14:22:31,3.0,19,34
3,Y,C_ID_faacef6d77,69,N,0,A,307,NaN,1,-0.671775,2018-02-01 09:42:32,1.0,9,19
4,Y,C_ID_e4d60467c2,48,N,1,B,703,NaN,1,-0.680791,2018-03-25 10:37:28,1.0,9,29


In [4]:
hist_transaction_data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
authorized_flag,26216,1,Y,26216,NaN,NaN,NaN,NaN,NaN,NaN,NaN
card_id,26216,26196,C_ID_8045bbaf71,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city_id,26216,NaN,NaN,NaN,137.962,102.334,-1,62,137,213,347
category_1,26216,2,N,26002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
installments,26216,NaN,NaN,NaN,0.331553,0.62298,-1,0,0,1,1
category_3,24034,2,A,13160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
merchant_category_id,26216,NaN,NaN,NaN,480.101,249.93,2,299,480,683,891
merchant_id,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
month_lag,26216,NaN,NaN,NaN,1.50122,0.500008,1,1,2,2,2
purchase_amount,26216,NaN,NaN,NaN,-0.591382,0.476551,-0.746893,-0.721363,-0.692061,-0.620482,21.7929


In [6]:
hist_transaction_data.isnull().sum()

authorized_flag             0
card_id                     0
city_id                     0
category_1                  0
installments                0
category_3               2182
merchant_category_id        0
merchant_id             26216
month_lag                   0
purchase_amount             0
purchase_date               0
category_2               1544
state_id                    0
subsector_id                0
dtype: int64

Observations from above:

1. state_id/ subsector_id /merchant_category_id/city_id has missing data, since i can see it has values which are - 1. We are going to leave them as is

2. installments has missing data , shown in the formats of both -1  we will take two strategies: replace both wiht mean and most common installment value to create two separate columns

3. month lag has only negative values, we need to look at it closely

4. category_3 / category_2 have both missing data. replacing it most common appearing data


In [7]:
hist_transaction_data.month_lag.value_counts()

2    13140
1    13076
Name: month_lag, dtype: int64

Since this is historical data with no merchant information attached, we will ignore the following columns:
    
    1. state_id
    2. city_id
    3. subsector_id
    4. merchant_category_id
    5. merchant_group_id
    6. category_1
    7. category_2

In [8]:
def prep_for_aggregate(table):
    
    table['authorized_flag'] = table.authorized_flag.map({'Y':1,'N':0})
    
   
    most_appearing = table.category_3.value_counts().idxmax()
    table.category_3.fillna(value = most_appearing, inplace = True)
    table['enc_category_3_A'] = (table.category_3=='A') +0
    table['enc_category_3_B'] = (table.category_3=='B') +0
    table['enc_category_3_C'] = (table.category_3=='C') +0
    table.drop(columns = ['category_3'], inplace = True)
    
    table['enc_purchase_date'] = panda.DatetimeIndex(table.purchase_date).astype(np.int64)*1e-9
    table.drop(columns = ['purchase_date'], inplace = True)

    mean_installments = np.mean(table.installments.values)
    most_common_installment = table.installments.value_counts().idxmax()
    
    table['installments_mean_replaced'] = table['installments']
    table.installments_mean_replaced.replace(-1, mean_installments, inplace= True)
    table.installments_mean_replaced.replace(999, mean_installments, inplace= True)
    
    table['installments_common_replaced'] = table['installments']
    table.installments_mean_replaced.replace(-1, most_common_installment, inplace= True)
    table.installments_mean_replaced.replace(999, most_common_installment, inplace= True)
    
    table.drop(columns = ['installments'], inplace = True)
    
    
    
    

In [9]:
prep_for_aggregate(hist_transaction_data)

In [10]:
hist_transaction_data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
authorized_flag,26216,NaN,NaN,NaN,1,0,1,1,1,1,1
card_id,26216,26196,C_ID_8045bbaf71,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city_id,26216,NaN,NaN,NaN,137.962,102.334,-1,62,137,213,347
category_1,26216,2,N,26002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
merchant_category_id,26216,NaN,NaN,NaN,480.101,249.93,2,299,480,683,891
merchant_id,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
month_lag,26216,NaN,NaN,NaN,1.50122,0.500008,1,1,2,2,2
purchase_amount,26216,NaN,NaN,NaN,-0.591382,0.476551,-0.746893,-0.721363,-0.692061,-0.620482,21.7929
category_2,24672,NaN,NaN,NaN,2.25961,1.52928,1,1,1,3,5
state_id,26216,NaN,NaN,NaN,10.8812,6.20336,-1,9,9,16,24


In [11]:
## we will take the 
## sum,mean,max of purchase amount, 
## sum,mean,max of month lag by card_id
## mean of authorized_flag,
## num of authorized ==1
## num of authorized ==0
## sum, mean, max of installments
## mean of category_1
## sum of enc_category_2* and enc_category_3*

def count_authorized_one(x):
    return x.eq(1).sum()
def count_authorized_zero(x):
    return x.eq(0).sum()
def count_category_1_one(x):
    return x.eq(1).sum()
def count_category_1_zero(x):
    return x.eq(0).sum()

def count_of_unique_cities(x):
    return x.nunique()

def count_of_unique_subsector_id(x):
    return x.nunique()

def count_of_unique_state_id(x):
    return x.nunique()

def most_common_merchant_id(x):
    c = Counter(x)
    return c.most_common()[0][0]


In [12]:
aggregate = {
    
'purchase_amount' : ['sum','mean','max'],
'month_lag' : ['sum','mean','max'] ,

'enc_category_3_A': ['sum'],
'enc_category_3_B': ['sum'],
'enc_category_3_C': ['sum'],
'installments_common_replaced': ['sum','mean','max'],
'installments_mean_replaced': ['sum','mean','max'],
'authorized_flag': ['mean', count_authorized_one, count_authorized_zero],

'enc_purchase_date' : ['mean','min','max',np.ptp],

    
}


In [13]:
group_by = hist_transaction_data.groupby(['card_id'])

In [14]:
len(hist_transaction_data)

26216

In [15]:
t = group_by.agg(aggregate)

In [16]:
t.head()

purchase_amount                     month_lag           \
                            sum      mean       max       sum mean max   
card_id                                                                  
C_ID_0001238066       -0.700326 -0.700326 -0.700326         1    1   1   
C_ID_0001506ef0       -0.715352 -0.715352 -0.715352         1    1   1   
C_ID_00032df08f       -0.640520 -0.640520 -0.640520         2    2   2   
C_ID_0004587331       -0.728876 -0.728876 -0.728876         1    1   1   
C_ID_000a9864df       -0.175900 -0.175900 -0.175900         1    1   1   

                enc_category_3_A enc_category_3_B enc_category_3_C  \
                             sum              sum              sum   
card_id                                                              
C_ID_0001238066                0                1                0   
C_ID_0001506ef0                1                0                0   
C_ID_00032df08f                0                1                0   
C_ID_0004587331                1                0                0   
C_ID_000a9864df                0                1                0   

                installments_common_replaced ...  installments_mean_replaced  \
                                         sum ...                         sum   
card_id                                      ...                               
C_ID_0001238066                            1 ...                         1.0   
C_ID_0001506ef0                            0 ...                         0.0   
C_ID_00032df08f                            1 ...                         1.0   
C_ID_0004587331                            0 ...                         0.0   
C_ID_000a9864df                            1 ...                         1.0   

                          authorized_flag                       \
                mean  max            mean count_authorized_one   
card_id                                                          
C_ID_0001238066  1.0  1.0               1                    1   
C_ID_0001506ef0  0.0  0.0               1                    1   
C_ID_00032df08f  1.0  1.0               1                    1   
C_ID_0004587331  0.0  0.0               1                    1   
C_ID_000a9864df  1.0  1.0               1                    1   

                                      enc_purchase_date                \
                count_authorized_zero              mean           min   
card_id                                                                 
C_ID_0001238066                     0      1.521891e+09  1.521891e+09   
C_ID_0001506ef0                     0      1.521239e+09  1.521239e+09   
C_ID_00032df08f                     0      1.515496e+09  1.515496e+09   
C_ID_0004587331                     0      1.519305e+09  1.519305e+09   
C_ID_000a9864df                     0      1.521823e+09  1.521823e+09   

                                    
                          max  ptp  
card_id                             
C_ID_0001238066  1.521891e+09  0.0  
C_ID_0001506ef0  1.521239e+09  0.0  
C_ID_00032df08f  1.515496e+09  0.0  
C_ID_0004587331  1.519305e+09  0.0  
C_ID_000a9864df  1.521823e+09  0.0  

[5 rows x 22 columns]

In [17]:
t_cols=['card_id']
t_cols.extend(['_'.join(i) for i in t.columns.tolist()])

t_cols

['card_id',
 'purchase_amount_sum',
 'purchase_amount_mean',
 'purchase_amount_max',
 'month_lag_sum',
 'month_lag_mean',
 'month_lag_max',
 'enc_category_3_A_sum',
 'enc_category_3_B_sum',
 'enc_category_3_C_sum',
 'installments_common_replaced_sum',
 'installments_common_replaced_mean',
 'installments_common_replaced_max',
 'installments_mean_replaced_sum',
 'installments_mean_replaced_mean',
 'installments_mean_replaced_max',
 'authorized_flag_mean',
 'authorized_flag_count_authorized_one',
 'authorized_flag_count_authorized_zero',
 'enc_purchase_date_mean',
 'enc_purchase_date_min',
 'enc_purchase_date_max',
 'enc_purchase_date_ptp']

In [18]:
t.reset_index(inplace=True)
t.columns = ['new_'+ i for i in t_cols]
t.head()

,new_card_id,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_month_lag_sum,new_month_lag_mean,new_month_lag_max,new_enc_category_3_A_sum,new_enc_category_3_B_sum,new_enc_category_3_C_sum,...,new_installments_mean_replaced_sum,new_installments_mean_replaced_mean,new_installments_mean_replaced_max,new_authorized_flag_mean,new_authorized_flag_count_authorized_one,new_authorized_flag_count_authorized_zero,new_enc_purchase_date_mean,new_enc_purchase_date_min,new_enc_purchase_date_max,new_enc_purchase_date_ptp
0,C_ID_0001238066,-0.700326,-0.700326,-0.700326,1,1,1,0,1,0,...,1.0,1.0,1.0,1,1,0,1.521891e+09,1.521891e+09,1.521891e+09,0.0
1,C_ID_0001506ef0,-0.715352,-0.715352,-0.715352,1,1,1,1,0,0,...,0.0,0.0,0.0,1,1,0,1.521239e+09,1.521239e+09,1.521239e+09,0.0
2,C_ID_00032df08f,-0.640520,-0.640520,-0.640520,2,2,2,0,1,0,...,1.0,1.0,1.0,1,1,0,1.515496e+09,1.515496e+09,1.515496e+09,0.0
3,C_ID_0004587331,-0.728876,-0.728876,-0.728876,1,1,1,1,0,0,...,0.0,0.0,0.0,1,1,0,1.519305e+09,1.519305e+09,1.519305e+09,0.0
4,C_ID_000a9864df,-0.175900,-0.175900,-0.175900,1,1,1,0,1,0,...,1.0,1.0,1.0,1,1,0,1.521823e+09,1.521823e+09,1.521823e+09,0.0


In [19]:
t.rename(columns = {'new_card_id':'card_id'}, inplace=True)
t.head()

,card_id,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_month_lag_sum,new_month_lag_mean,new_month_lag_max,new_enc_category_3_A_sum,new_enc_category_3_B_sum,new_enc_category_3_C_sum,...,new_installments_mean_replaced_sum,new_installments_mean_replaced_mean,new_installments_mean_replaced_max,new_authorized_flag_mean,new_authorized_flag_count_authorized_one,new_authorized_flag_count_authorized_zero,new_enc_purchase_date_mean,new_enc_purchase_date_min,new_enc_purchase_date_max,new_enc_purchase_date_ptp
0,C_ID_0001238066,-0.700326,-0.700326,-0.700326,1,1,1,0,1,0,...,1.0,1.0,1.0,1,1,0,1.521891e+09,1.521891e+09,1.521891e+09,0.0
1,C_ID_0001506ef0,-0.715352,-0.715352,-0.715352,1,1,1,1,0,0,...,0.0,0.0,0.0,1,1,0,1.521239e+09,1.521239e+09,1.521239e+09,0.0
2,C_ID_00032df08f,-0.640520,-0.640520,-0.640520,2,2,2,0,1,0,...,1.0,1.0,1.0,1,1,0,1.515496e+09,1.515496e+09,1.515496e+09,0.0
3,C_ID_0004587331,-0.728876,-0.728876,-0.728876,1,1,1,1,0,0,...,0.0,0.0,0.0,1,1,0,1.519305e+09,1.519305e+09,1.519305e+09,0.0
4,C_ID_000a9864df,-0.175900,-0.175900,-0.175900,1,1,1,0,1,0,...,1.0,1.0,1.0,1,1,0,1.521823e+09,1.521823e+09,1.521823e+09,0.0


In [20]:
t.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
card_id,26196,26196,C_ID_a60c0bf58e,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
new_purchase_amount_sum,26196,NaN,NaN,NaN,-0.591834,0.477032,-1.49111,-0.721363,-0.692211,-0.620516,21.7929
new_purchase_amount_mean,26196,NaN,NaN,NaN,-0.591438,0.476502,-0.746893,-0.721363,-0.692061,-0.620482,21.7929
new_purchase_amount_max,26196,NaN,NaN,NaN,-0.591342,0.476703,-0.746893,-0.721363,-0.692061,-0.620441,21.7929
new_month_lag_sum,26196,NaN,NaN,NaN,1.50237,0.502061,1,1,2,2,4
new_month_lag_mean,26196,NaN,NaN,NaN,1.50126,0.500008,1,1,2,2,2
new_month_lag_max,26196,NaN,NaN,NaN,1.50126,0.500008,1,1,2,2,2
new_enc_category_3_A_sum,26196,NaN,NaN,NaN,0.585662,0.493623,0,0,1,1,2
new_enc_category_3_B_sum,26196,NaN,NaN,NaN,0.415102,0.493291,0,0,0,1,2
new_enc_category_3_C_sum,26196,NaN,NaN,NaN,0,0,0,0,0,0,0


In [21]:
del hist_transaction_data

In [22]:
t.to_csv('new_without_merchant_aggregated.csv', index = False)